Eddy Tamayo GEOG 458 Arcpy lab

For each county in Washington, create a GeoJSON file whose filename is the county name and whose contents are the polygons for the block groups within that county. This means that you will end up with as many GeoJSON files as there are counties in WA. Each such file should be named after a county and contain only the block groups from that county.

In [1]:
#Setting up files to work with
import sys
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\bin')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\arcpy')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\ArcToolbox\\Scripts')
import arcpy

arcpy.env.overwriteOutput = True    #arcpy environment is being set in order to code

In [26]:
#I am setting up a copy of my shapefile while also creating fc = my saep_bg10 shapefile / I decided to work on my C drive
from arcpy import env
env.workspace = "C:/arcpy458/saep_bg10"
saep = "saep_bg10.shp"

arcpy.management.CopyFeatures(
    "C:\\arcpy458\\saep_bg10.shp",
    "C:\\arcpy458\\saep_bg10_copy.shp") # copies features from the input feature class to a new feature class.

fc= "C:\\arcpy458\\saep_bg10_copy.shp"  #setting up a variable that is equal to the feature class file within my workspace

In [32]:
fc= "C:\\arcpy458\\saep_bg10_copy.shp"
arcpy.AddField_management(fc, "FIPSCounty","TEXT")
# I am adding the field FIPSCounty to my saep_bg10 file / The type of measurement is a TEXT
# adding a new field to my feature class -attribute table now has FIPSCounty listed as a field with no values


<Result 'C:\\arcpy458\\saep_bg10_copy.shp'>

In [33]:
fc= "C:\\arcpy458\\saep_bg10_copy.shp"
arcpy.AddField_management(fc, "CountyName","TEXT")
#adding CountyNAme to my feature class - attribute table now has CountyName with measurement value as a TEXT

<Result 'C:\\arcpy458\\saep_bg10_copy.shp'>

In [42]:
table = "C:\\arcpy458\\WashingtonFIPS.dbf" # I need to create a variable that would be equal to my WashingtonFIPS table
# I can then use table to do a join field within ArcPy
# "table" creates a path for the table file i will use

In [44]:
join_table = "C:\\arcpy458\\WashingtonFIPS.dbf"
arcpy.JoinField_management (fc,"COUNTYFP10",table,"FIPSCounty",["CountyName"])
#I am joining the contents of a table to another table based on a common attribute field
#CountyFP10 and FIPSCounty are the similar attribute fields
#the input table is updated to contain the fields from the WashingtonFIPS.dbf table

<Result 'C:\\arcpy458\\saep_bg10_copy.shp'>

In [52]:
#creates a feature class within my arcpy458 folder
arcpy.CreateFeatureclass_management("C:\\arcpy458\\","new_blocks.shp","polygon","C:\\arcpy458\\saep_bg10_copy.shp")
# this management tool created an empty feature class -However, there is no geometry yet for this shapefile


<Result 'C:\\arcpy458\\new_blocks.shp'>

In [71]:
my_list = []
cursor = arcpy.da.SearchCursor(table, ["CountyName"])
for row in cursor:
                               # a search cursor is needed to look up all the county names within the attribute table.
    my_list.append(row[0])     # I used a for loop in order to create a list of all the counties 
                               # within the table. My_list append is created from the CountyName field
                               #my_list is set up so the program knows where my list is going to
    
                                # Printed a list of all the counties in alphebetical order
                                # for every county in the list, I must then create a shapefile
                                # for it. For every row in the table  if county name = county, i insert that row
del cursor

print my_list

[u'Adams', u'Asotin', u'Benton', u'Chelan', u'Clallam', u'Clark', u'Columbia', u'Cowlitz', u'Douglas', u'Ferry', u'Franklin', u'Garfield', u'Grant', u'Grays Harbor', u'Island', u'Jefferson', u'King', u'Kitsap', u'Kittitas', u'Klickitat', u'Lewis', u'Lincoln', u'Mason', u'Okanogan', u'Pacific', u'Pend Oreille', u'Pierce', u'San Juan', u'Skagit', u'Skamania', u'Snohomish', u'Spokane', u'Stevens', u'Thurston', u'Wahkiakum', u'Walla Walla', u'Whatcom', u'Whitman', u'Yakima']


In [75]:
for County in my_list:
    arcpy.FeatureClassToFeatureClass_conversion(fc, "C:\\arcpy458\\", str(County), '"CountyNa_1" = ' + "'%s'" %County)
                 #This creates a shapfile for every CountyName within the attribute table.                                
                                                 

In [6]:
ogr_dir = 'OSGeo4W' 

from subprocess import call
import os

os.environ["GDAL_DATA"] = "C:\\OSGeo4W64\\share\\gdal"
os.environ["GDAL_DRIVER_PATH"] = "C:\\OSGeo4W64\\bin\\gdalplugins"
os.environ["PROJ_LIB"] = "C:\\OSGeo4W64\\share\\proj"
os.environ["PATH"] = "C:\\OSGeo4W64\\bin;"+os.environ["PATH"]+";C:\\OSGeo4W64\\apps\\msys\\bin;C:\\OSGeo4W64\\apps\\Python27\\Scripts"
#This sets up the environment variables for GDAL/OGR
#Producing GEOJSON files out of ArcPy Data
#This code helps to convert to geojson files 

In [77]:
for County in my_list:
    call(['C:\\OSGeo4W64\\bin\\ogr2ogr',    # converting shapefiles into GeoJSON files 
          '-f','GeoJSON','-t_srs','WGS84',  # EPSG edited to 2927
          '-s_srs','EPSG:2927',
          'C:\\arcpy458\\'+ County +'.geojson',
          'C:\\arcpy458\\'+ County +'.shp'])
    # For every county in my list, there is a geojson file being created for every county that
    # has a shapefile. "county" is the name for all counties in my_list. This is considered a for loop
    # to search for every county and establish a geojson file 

Print out a ranked list, in descending order, of the ten largest total populations in 2017 of counties in Washington, according to this dataset.

In [110]:
pop_list = []
for County in my_list:    # creating a pop list from county in my list to get the POP2017 values needed.
    pop = 0               #Search cursor is needed to find which field in the attribute table I
                          # will pull out information from. POP2017 field will be used
    cursor = arcpy.da.SearchCursor(fc, ["POP2017"], '"CountyNa_1" = ' + "'%s'" % County)

    for row in cursor:
        pop += row[0]
    pop_list.append(pop)   #This is considered a loop where it is looking for POP2017 values

pop_list.sort(reverse=True) # distinguishes from ascending and descending in terms of the order we want the pop values in.
i = 0
while i < 10:
    print pop_list[i]  # this while statement prints out the values in the correct order
    i += 1
    

2153700.0
859400.0
789400.0
499800.0
471000.0
276900.0
264300.0
253000.0
216300.0
193500.0


This lab took me about 14 hours to complete. This was a good introduction in terms of actually putting in to use the material we have learned so far. Their were moments of frustration but this lab made me learn useful python scripting skills. I initially did not know where to begin considering I dont have prior coding/scripting experience so it did feel a little overwhelming at first. I incorporated loops, nested loops, etc. I started off by setting up my variables for the feature classes as well as my WashingtonFIPS.dbf table.I then incorporated those variables to come up with results using cursors. The search cursors were helpful in terms of finding the relevant fields and values to answer both problems. I spoke with Luis about this lab and discussed how to tackle this lab in an efficient manner.